In [ ]:
# References:
# Data from:
# https://github.com/Werneror/Poetry.git

In [60]:
!rm Poetry/ -rf

In [1]:
!git clone https://github.com/Werneror/Poetry.git

Cloning into 'Poetry'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 152 (delta 4), reused 9 (delta 2), pack-reused 139
Receiving objects: 100% (152/152), 123.55 MiB | 10.95 MiB/s, done.
Resolving deltas: 100% (84/84), done.
Updating files: 100% (39/39), done.


In [28]:
import pandas as pd

In [29]:
data_dir = "./Poetry/"
%ls $data_dir

LICENSE*       宋_1.csv*      明_4.csv*          近现代.csv*
README.md*     宋_2.csv*      明末清初.csv*      近现代末当代初.csv*
cipai_2.txt*   宋_3.csv*      民国末当代初.csv*  金.csv*
scripts/       宋_4.csv*      汉.csv*            金末元初.csv*
元.csv*        宋末元初.csv*  清_1.csv*          隋.csv*
元末明初.csv*  宋末金初.csv*  清_2.csv*          隋末唐初.csv*
先秦.csv*      当代.csv*      清末民国初.csv*    魏晋.csv*
南北朝.csv*    明_1.csv*      清末近现代初.csv*  魏晋末南北朝初.csv*
唐.csv*        明_2.csv*      秦.csv*
唐末宋初.csv*  明_3.csv*      辽.csv*


In [30]:
data_fn = data_dir + "唐.csv"
%ls $data_fn

./Poetry/唐.csv*


In [31]:
df = pd.read_csv(data_fn)
df.head()

,题目,朝代,作者,内容
0,忆秦娥,唐,冯延巳,风淅淅。夜雨连云黑。滴滴。窗外芭蕉灯下客。除非魂梦到乡国。免被关山隔。忆忆。一句枕前争忘得。
1,送兄,唐,七岁女子,别路云初起，离亭叶正飞。所嗟人异雁，不作一行归。
2,再赠,唐,上元夫人,弄玉有夫皆得道，刘纲兼室尽登仙。君能仔细窥朝露，须逐云车拜洞天。
3,留别,唐,上元夫人,萧郎不顾凤楼人，云涩回车泪脸新。愁想蓬瀛归去路，难窥旧苑碧桃春。
4,赠封陟,唐,上元夫人,谪居蓬岛别瑶池，春媚烟花有所思。为爱君心能洁白，愿操箕帚奉屏帏。


In [32]:
batch_size = 64 # B, batch size
block_size = 48 # T, context len for poem is shorter, to set to 48
#vocab_size = len(t2i.keys())
nn_emb_size = 64 # nn_emb
n_head = 16
n_layers = 8

device = "cuda"

In [33]:
# drop invalid char "{", '□', '《'
df = df[df['内容'].str.contains("{") == False]
df = df[df['内容'].str.contains('□') == False]
df = df[df['内容'].str.contains('《') == False]

# drop context len > block_size
#df = df[df['内容'].str.len()<=block_size]

In [34]:
rare_chars = [
'㒿',
 '㗫',
 '㗻',
 '㘭',
 '㟅',
 '㟍',
 '㠔',
 '㩳',
 '㪍',
 '㪷',
 '㬠',
 '㭊',
 '㴩',
 '㵳',
 '㶁',
 '㸌',
 '㸙',
 '㸦',
 '㹀',
 '㹞',
 '㾕',
 '䆉',
 '䆗',
 '䋏',
 '䍥',
 '䍦',
 '䍲',
 '䍽',
 '䏑',
 '䏶',
 '䐑',
 '䑳',
 '䒠',
 '䔩',
 '䔫',
 '䖘',
 '䖟',
 '䗈',
 '䗫',
 '䙰',
 '䛏',
 '䜝',
 '䨴',
 '䩋',
 '䪻',
 '䫻',
 '䭀',
 '䭃',
 '䭔',
 '䯀',
 '䱐',
 '䲺',
 '䳒',
 '䴏',
 '䴥',
 '䴵',
 '䴺',
 '䶎']

for c in rare_chars:
    df = df[df['内容'].str.contains(c) == False]

In [35]:
df[df['内容'].str.contains(c) == True]

,题目,朝代,作者,内容


In [36]:
all_data = df['内容'].tolist()
all_data[:5]

['风淅淅。夜雨连云黑。滴滴。窗外芭蕉灯下客。除非魂梦到乡国。免被关山隔。忆忆。一句枕前争忘得。',
 '别路云初起，离亭叶正飞。所嗟人异雁，不作一行归。',
 '弄玉有夫皆得道，刘纲兼室尽登仙。君能仔细窥朝露，须逐云车拜洞天。',
 '萧郎不顾凤楼人，云涩回车泪脸新。愁想蓬瀛归去路，难窥旧苑碧桃春。',
 '谪居蓬岛别瑶池，春媚烟花有所思。为爱君心能洁白，愿操箕帚奉屏帏。']

In [37]:
data = "".join(all_data)
data[:50]

'风淅淅。夜雨连云黑。滴滴。窗外芭蕉灯下客。除非魂梦到乡国。免被关山隔。忆忆。一句枕前争忘得。别路云初'

In [38]:
data.index("$")

ValueError: substring not found

In [39]:
vocab = sorted(list(set(data)))
# add padding
vocab = ["$"] + vocab

In [40]:
data.index('《')

ValueError: substring not found

In [41]:
data[72306:72316]

'馆晴。晚来因废卷，行'

In [42]:
vocab[0:500]

['$',
 '?',
 '◇',
 '、',
 '。',
 '「',
 '」',
 '㑻',
 '㓤',
 '㓨',
 '㔩',
 '㕙',
 '㕭',
 '㖇',
 '㖷',
 '㗀',
 '㗧',
 '㘈',
 '㙞',
 '㙧',
 '㙻',
 '㜕',
 '㟏',
 '㟼',
 '㠁',
 '㠢',
 '㠥',
 '㡊',
 '㤝',
 '㤞',
 '㥄',
 '㦎',
 '㦨',
 '㦬',
 '㧞',
 '㨖',
 '㨝',
 '㩧',
 '㩻',
 '㪇',
 '㯕',
 '㰂',
 '㰕',
 '㰤',
 '㰹',
 '㱔',
 '㱥',
 '㲉',
 '㲚',
 '㲪',
 '㴲',
 '㵝',
 '㹠',
 '㺄',
 '㺑',
 '㺗',
 '㺠',
 '㺦',
 '㻞',
 '㼐',
 '㼚',
 '㽅',
 '㽬',
 '㾌',
 '㾓',
 '㿉',
 '䀣',
 '䀨',
 '䁠',
 '䂍',
 '䂓',
 '䃂',
 '䃔',
 '䃧',
 '䃸',
 '䅎',
 '䆕',
 '䆛',
 '䆷',
 '䇹',
 '䈉',
 '䈴',
 '䉛',
 '䉦',
 '䌨',
 '䌰',
 '䍡',
 '䑔',
 '䔿',
 '䕓',
 '䕭',
 '䕹',
 '䗁',
 '䗱',
 '䘧',
 '䘨',
 '䙀',
 '䛠',
 '䜕',
 '䝟',
 '䞋',
 '䟃',
 '䟐',
 '䢇',
 '䤨',
 '䥓',
 '䥫',
 '䦪',
 '䦰',
 '䦱',
 '䨟',
 '䨥',
 '䪗',
 '䪜',
 '䪥',
 '䫇',
 '䫜',
 '䬓',
 '䬔',
 '䬘',
 '䬴',
 '䮂',
 '䮧',
 '䯔',
 '䯗',
 '䯱',
 '䰀',
 '䱙',
 '䱜',
 '䱬',
 '䴊',
 '䶉',
 '一',
 '丁',
 '七',
 '万',
 '丈',
 '三',
 '上',
 '下',
 '不',
 '与',
 '丐',
 '丑',
 '专',
 '且',
 '丕',
 '世',
 '丘',
 '丙',
 '业',
 '丛',
 '东',
 '丝',
 '丞',
 '两',
 '严',
 '丧',
 '个',
 '丫',
 '中',
 '丰',
 '丱',
 '串',
 '丳',
 '临',
 '丸'

In [43]:
i2t = {k:v for k,v in enumerate(vocab)}
{k:i2t[k] for k in list(i2t.keys())[0:5]}

{0: '$', 1: '?', 2: '◇', 3: '、', 4: '。'}

In [44]:
t2i = {v:k for k,v in enumerate(vocab)}
{k:t2i[k] for k in list(t2i.keys())[0:5]}

{'$': 0, '?': 1, '◇': 2, '、': 3, '。': 4}

In [45]:
def encode(s): return [ t2i[c] for c in s]
encode("瓶怒")

[3897, 1900]

In [46]:
def decode(l): return "".join([ i2t[i] for i in l])
decode([1,3])

'?、'

In [47]:
len(vocab)

7475

In [48]:
len(t2i.keys())

7475

In [49]:
import pickle
meta = { "t2i": t2i, "i2t": i2t }
with open('meta.pkl', 'wb') as f:
    pickle.dump(meta,f)

In [50]:
%ls

CN_tokenizer.ipynb*       GPTModel_MLP_v2.ipynb*    GPTModel_v3_fail.ipynb*
CN_tokenizer_mini.ipynb*  GPTModel_atten_v1.ipynb*  Poetry/
GPTModel.ipynb*           GPTModel_atten_v2.ipynb*  meta.pkl*
GPTModel_MLP.ipynb*       GPTModel_atten_v3.ipynb*  model_v1.pkl*


In [51]:
with open('meta.pkl','rb') as f:
    meta = pickle.load(f)
vocab_size = len(meta['t2i'].keys())
vocab_size

7475

In [52]:
vocab = meta['t2i'].keys()

In [53]:
len(vocab)

7475